In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

# Computing the Conjunctive Normal Form in First Order Logic

In order to convert a formula $f$ from first order logic into a set of clauses that is satisfiable if and only if $f$ is satisfiable,
we have to perform the following steps in order:
- eliminate biconditionals,
- eliminate conditionals,
- transform the formula into *negation normal form*,
  i.e. we push the negation symbol inwards,
- rename bound variables to avoid clashes, 
- transform the formula into *prenex normal form*,
  i.e. we move the quantifieres outside,
  
- eliminate existential quantifiers by *skolemizing* the formula, and
- transform the formula into *clauses* in set notation.

When converting formulas into conjunctive normal form, we <u>assume</u> that the formulas are 
*pure*, where we define a formula $f$ as *pure* if all quantifiers appearing in $f$ bind **different** variables.  For example, the formula
$$ \bigl(\forall x: p(x)\bigr) \vee \bigl(\forall x: q(x)\bigr)$$
is **not** *pure*, because there are two different universal quantifiers that both bind the same variable $x$.  We can rewrite this formulas as a *pure* formula by *renaming* all occurrences of $x$ that are bound by the second quantifier as follows:
$$ \bigl(\forall x: p(x)\bigr) \vee \bigl(\forall y: q(y)\bigr)$$

## Auxilliary Functions

Formulas are represented as nested tuples.  In order to convert a string into a nested tuple we use the <tt>LogicParser</tt> that is found in the module <tt>folParser</tt>.  Our parser distinguishes variables and function symbol as follows:
- A word starting with a *lower* case letter is interpreted as a *variable*.
- A word starting with an *upper* case letter is assumed to be a *function* or *predicate symbol*.

In [ ]:
%%capture
%run FOL-Parser.ipynb

The function $\texttt{parse}(s)$ takes a string $s$ which is a formula from first order logic and turns this string into a 
*nested tuple*.

In [ ]:
def parse(s):
    return LogicParser(s).parse()

For testing purposes, the following formula is used.  This formula specifies the notion of a *grandparent*.

In [ ]:
s  = '∀g:∀c:(Grandparent(g, c) ↔ ∃p: (Parent(g, p) ∧ Parent(p, c)))'
f1 = parse(s)
f1

The function $\texttt{apply}(t, σ)$ takes an object $t$ and a *variable substitution* $\sigma$ which is represented as a dictionary of the form $\{x_1: s_1, \cdots, x_n:s_n\}$ and replaces every occurrence of the variable $x_i$ in the object $t$ with the corresponding term $s_i$.  The object $t$ is either 
 - a term, 
 - a formula from first order logic (henceforth abbreviated as *FOL*), 
 - a clause (represented as a `frozenset` of literals), or 
 - a set of clauses.

In [ ]:
def apply(t, σ):
    'Apply the substitution σ to the term t.'
    if isinstance(t, (set, frozenset)):      # t is a set of clauses or a frozenset of literals
        return { apply(c, σ) for c in t }
    if isinstance(t, str):                   # t is a variable
        if t in σ:
            return σ[t]
        else:
            return t
    else:           
        f, *ts = t  # at this point, t can only be an fol formula or a term, 
        return (f,) + tuple(apply(s, σ) for s in ts)

The assignment `f, *ts = t` shown above uses so called *extended iterable unpacking*.  The code below shows an example how this works.

In [ ]:
f, *ts = ('parent', 'hugo', 'gustav')
f, ts

In [ ]:
f1

In [ ]:
apply(f1, { 'g': 'x', 'p': 'y', 'c': 'z' })

The function $\texttt{boundVariables}(f)$ computes the set of variables that are *bound* in the formula $f$. 

In [ ]:
def boundVariables(f):
    match f:
        case Q, x, g if Q in { '∀', '∃' }:
            return { x } | boundVariables(g)
        case ('⊤',):
            return set()
        case ('⊥',):
            return set()
        case '¬', g:
            return boundVariables(g)
        case op, g, h if op in ('∧', '∨', '→', '↔'):
            return boundVariables(g) | boundVariables(h)
        case _:
            return set()  # f must be an atomic formula

In [ ]:
f1

In [ ]:
boundVariables(f1)

The function `allVariables` computes the set of all variables that occur in terms inside `f`. The object 
`f` is either a formula or a term.

In [ ]:
def allVariables(f):
    match f:
        case Q, x, g if Q in ('∀', '∃'):
            return { x } | allVariables(g)
        case ('⊤',):
            return set()
        case ('⊥',):
            return set()
        case '¬', g:
            return allVariables(g)
        case op, g, h if op in ('∧', '∨', '→', '↔'):
            return allVariables(g) | allVariables(h)
        case x if isinstance(x, str):  # f is a variable
            return { x }
        case _, *args:
            return { x for t in args 
                       for x in allVariables(t) 
                   }

In [ ]:
f1

In [ ]:
allVariables(f1)

In [ ]:
g1 = ('↔', ('Grandparent', 'g', 'c'),
           ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c')))
     )

In [ ]:
allVariables(g1)

Below we import the module `string` because it provides a definition of all lower case characters.

In [ ]:
import string
string.ascii_lowercase

In [ ]:
set(string.ascii_lowercase)

The function $\texttt{renameBoundVariables}(f)$ takes a first order formula $f$ and replaces all bound variables by **new** variables.  This only works if the set of characters `set(string.ascii_lowercase)` has enough characters that do not already occur in $f$.  This approach would not be good enough for a production quality program,
but for the case of a demonstration it is sufficient.  The alternative would be to rename the variables as `x1`, `x2`, `x3`, $\cdots$, but that becomes unreadable very fast.

In [ ]:
def renameBoundVariables(f):
    BoundVs = boundVariables(f)
    NewVars = set(string.ascii_lowercase) - allVariables(f)
    NewVars = sorted(list(NewVars))
    sigma   = { x: NewVars[i] for (i, x) in enumerate(BoundVs) }
    return apply(f, sigma)

In [ ]:
list(enumerate(['a', 'b', 'c']))

In [ ]:
f1

In [ ]:
renameBoundVariables(f1)

## Elimination Biconditionals

The function $\texttt{eliminateBiconditional}(f)$ takes a formula $f$ from first order logic and eliminates all occurrences of the operator '↔' from this formula.  This is done by using the following equivalence:
$$ (f \leftrightarrow g) \;\Leftrightarrow\; (f \rightarrow g) \wedge (g \rightarrow f) $$
In order to ensure that the resulting formula is <em style="color:blue">pure</em>, we have to rename the bound variables in the formula $g \rightarrow f$.

In [ ]:
def eliminateBiconditional(f):
    "Eliminate the logical operator '↔' from the formula f."
    match f:
        case '↔', g, h:
            ge = eliminateBiconditional(g)
            he = eliminateBiconditional(h)
            return ('∧', ('→', ge, he), renameBoundVariables(('→', he, ge)))
        case ('⊤', ):
            return f
        case ('⊥', ):
            return f
        case '¬', g:
            return ('¬', eliminateBiconditional(g))
        case op, g, h if op in ('∧', '∨', '→'):
            return (op, eliminateBiconditional(g), eliminateBiconditional(h))
        case Q, x, g if Q in ('∀', '∃'):
            return (Q, x, eliminateBiconditional(g))
        case _:   # f must be an atomic formula
            return f              

In [ ]:
f1

In [ ]:
f2 = eliminateBiconditional(f1)
f2

## Eliminating Conditionals

The function $\texttt{eliminateConditional}(f)$ takes a formula $f$ from first order logic and eliminates all occurrences of the operator '→' from this formula.  This is done by using the following equivalence:
$$ (f \rightarrow g) \;\Leftrightarrow\; (\neg f \vee g) $$
The implementation of this function is similar to the implementation of the function `eliminateConditional` that we had used in propositional logic.

In [ ]:
def eliminateConditional(f):
    "Eliminate the logical operator '→' from f."
    match f:
        case '→', g, h:
            return ('∨', ('¬', eliminateConditional(g)), eliminateConditional(h))
        case ('⊤', ):
            return f
        case ('⊥', ):
            return f
        case '¬', g:
            return ('¬', eliminateConditional(g))
        case op, g, h if op in ('∧', '∨'):
            return (op, eliminateConditional(g), eliminateConditional(h))
        case Q, x, g if Q in ('∀', '∃'):
            return (Q, x, eliminateConditional(g))
        case _:  # f must be an atomic formula
            return f 

In [ ]:
f2

In [ ]:
f3 = eliminateConditional(f2)
f3

## Negation Normal Form

The function $\texttt{nnf}(f)$ computes the <em style="color:blue;">negation normal form</em> of $f$, while $\texttt{neg}(f)$ computes the *negation normal form* of $\neg f$.  The expression $\texttt{nnf}(f)$ is defined recursively as follows:
<ol>
    <li> $\texttt{nnf}(\neg \texttt{F}) = \texttt{neg}(\texttt{F})$, </li>
    <li> $\texttt{nnf}(\texttt{F}_1 \wedge \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \wedge \texttt{nnf}(\texttt{F}_2)$,</li>
    <li> $\texttt{nnf}(\texttt{F}_1 \vee \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \vee \texttt{nnf}(\texttt{F}_2)$.</li>
    <li> $\texttt{nnf}(\forall x: F) = \forall x: \texttt{nnf}(\texttt{F})$.</li>
    <li> $\texttt{nnf}(\exists x: F) = \exists x: \texttt{nnf}(\texttt{F})$.</li>
</ol>

In [ ]:
def nnf(f):
    "Compute the negation normal form of f."
    match f:
        case ('⊤',):
            return f
        case ('⊥', ):
            return f
        case '¬', g:
            return neg(g)
        case op, g, h if op in ('∧', '∨'):
            return op, nnf(g), nnf(h)
        case Q, x, g if Q in ('∀', '∃'):
            return Q, x, nnf(g)
        case _: # f must be atomic 
            return f                     

The auxiliary function $\texttt{neg}$ is also defined recursively:
<ol>
    <li> $\texttt{neg}(p) = \texttt{nnf}(\neg p) = \neg p$ for all propositional variables $p$,</li>
    <li> $\texttt{neg}(\neg F) = \texttt{nnf}(\neg \neg F) = \texttt{nnf}(F)$,</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \wedge F_2 \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \wedge F_2)\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \vee \neg F_2\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \vee \texttt{nnf}\bigl(\neg F_2\bigr) \\[0.1cm]
       = & \texttt{neg}(F_1) \vee \texttt{neg}(F_2).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \wedge F_2 \bigr) = \texttt{neg}(F_1) \vee \texttt{neg}(F_2)$.</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \vee F_2 \bigr)        \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \vee F_2) \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \wedge \neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \wedge \texttt{nnf}\bigl(\neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{neg}(F_1) \wedge \texttt{neg}(F_2). 
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \vee F_2 \bigr) = \texttt{neg}(F_1) \wedge \texttt{neg}(F_2)$.</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(\forall x: F \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg \forall x: F\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\exists x: \neg F\bigr) \\[0.1cm]
       = & \exists x: \texttt{nnf}(\neg F)           \\[0.1cm]
       = & \exists x: \texttt{neg}(F).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(\forall x: F \bigr) = \exists x: \texttt{neg}(F)$.</li>
      <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(\exists x: F \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg \exists x: F\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\forall x: \neg F\bigr) \\[0.1cm]
       = & \forall x: \texttt{nnf}(\neg F)           \\[0.1cm]
       = & \forall x: \texttt{neg}(F).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(\exists x: F \bigr) = \forall x: \texttt{neg}(F)$.</li>
</ol>

In [ ]:
def neg(f):
    "Compute the negation normal form of ¬f."
    match f:
        case ('⊤', ):
            return ('⊥', )
        case ('⊥', ):
            return ('⊤', )
        case '¬', g:
            return nnf(g)
        case '∧', g, h:
            return '∨', neg(g), neg(h)
        case '∨', g, h:
            return '∧', neg(g), neg(h)
        case '∀', x, g:
            return '∃', x, neg(g)
        case '∃', x, g:
            return '∀', x, neg(g)
        case _: # f must be atomic here
            return '¬', f              

In [ ]:
f3

In [ ]:
f4 = nnf(f3)
f4

## Prenex Normal Form

In the following we assume that all quantifiers that occur in a formula bind **different** variables, i.e. we assume that the formulas are *pure*.  If this assumption is not satisfied, then the functions given below will produce <u>garbage</u>.

A *quantifier tuple* is a tuple of the following form:
$$ (Q_1, x_1, \cdots, Q_n, x_n) $$
Here, the $Q_i$ denote quantifiers, i.e. we have $Q_i \in \{\forall, \exists\}$, while the $x_i$ are variables.  The function $\texttt{mergeQuantifiers}(T_1, T_2)$ takes two quantifier tuples $T_1$ and $T_2$ as arguments and merges them into a new quantifier tuple such that the relative order of the quantifiers remains the same, i.e. if both $Q_1, x_1$ and $Q_2, x_2$ occur in $T_1$ and $Q_1, x_1$ occurs before $Q_2, x_2$, then $Q_1, x_1$ will occur before $Q_2, x_2$ in the result.

In [ ]:
def mergeQuantifiers(Q1, Q2):
    if Q1 == ():
        return Q2
    if Q2 == ():
        return Q1
    if Q1[0] == '∃':  # extract existential quantifiers first
        return Q1[:2] + mergeQuantifiers(Q1[2:], Q2)
    if Q2[0] == '∃':
        return Q2[:2] + mergeQuantifiers(Q1, Q2[2:])
    return Q1[:2] + mergeQuantifiers(Q1[2:], Q2)

In [ ]:
mergeQuantifiers(('∀', 'x', '∃', 'y'), ('∃', 'u', '∀', 'v'))

Given a formula $f$, the function $\texttt{extractQuantifiers}(f)$ returns a pairs $(T, m)$, where $T$ is a quantifier tuple and $m$ is the <em style="color:blue;">matrix</em> of the formula $f$, where the matrix of a formula is defined as the part that remains when all quantifiers have been extracted.

In [ ]:
def extractQuantifiers(f):
    match f:
        case (op,) if op in ('⊤', '⊥'):
            return (), f
        case '¬', _:
            return (), f
        case op, g, h if op in ('∧', '∨'):
            qg, gm = extractQuantifiers(g)
            qh, hm = extractQuantifiers(h)
            # this works because f is pure
            return mergeQuantifiers(qg, qh), (op, gm, hm)
        case Q, x, g if Q in ('∀', '∃'):
            qg, gm  = extractQuantifiers(g)
            return (Q, x) + qg, gm
        case _: # f must be atomic 
            return (), f             

In [ ]:
f4

In [ ]:
Qs, f5 = extractQuantifiers(f4)
Qs, f5

Given a qantifier tuple $\texttt{Qs}$ and a matrix $m$, the call $\texttt{attachQuantifiers}(Qs, m)$ combines the quantifiers $\texttt{Qs}$ and the matrix $m$ into a quantified formula.

In [ ]:
def attachQuantifiers(Qs, m):
    if Qs == ():
        return m
    Q, x, *Qr = Qs
    return (Q, x, attachQuantifiers(tuple(Qr), m))

In [ ]:
Qs

In [ ]:
f5

In [ ]:
f6 = attachQuantifiers(Qs, f5)
f6

## Skolemization (Eliminating Existential Quantifiers)

The variable $\texttt{skolemCounter}$ is a global variable that is needed to create unique Skolem constants.  

In [ ]:
skolemCounter = 0

In [ ]:
def skolemConstant():
    global skolemCounter
    skolemCounter += 1
    return 'sk' + str(skolemCounter)

The function $\texttt{skolemize}(f, \texttt{Vs})$ takes a formula $f$ and a tuple of variables $\texttt{Vs}$ and 
<em style="color:blue">skolemizes</em> the formula $f$, i.e. it replaces all existentially quantified variables by appropriate <em style="color:blue">Skolem functions</em>.  The tuple $\texttt{Vs}$ is a tuple of variables that are 
assumed to be universally quantified.  The formula $f$ is assumed to be in <em style="color:blue">prenex normal form</em>.

For skolemization to work correctly, we have to assume that 
<font size="4" style="color:darkgreen; size:125%">$f$ does not contain free variables</font>!

In [ ]:
def skolemize(f, Vs):
    match f:
        case '∃', x, g:
            t = (skolemConstant(),) + Vs 
            σ = { x: t }
            return skolemize(apply(g, σ), Vs)
        case '∀', x, g:
            return skolemize(g, Vs + (x,))
        case _: # at this point f cannot contain a quantifier 
            return f                             

In [ ]:
f6

In [ ]:
f7 = skolemize(f6, ())
f7

## Conversion to Clauses

The function $\texttt{cnf}(f)$ takes a <em style="color:blue">skolemized</em> formula $f$ from first order logic that is in <em style="color:blue">negation normal form</em> and returns the <em style="color:blue">conjunctive normal form</em> of $f$ in <em style="color:blue">set notation</em>.  This works the same way as in propositional logic.

In [ ]:
def cnf(f):
    match f:
        case ('⊤', ):
            return set()
        case ('⊥', ):
            return {frozenset()}
        case '¬', _:
            return { frozenset({f}) }
        case '∧', g, h:
            return cnf(g) | cnf(h)
        case '∨', g, h:
            return { k1 | k2 for k1 in cnf(g) for k2 in cnf(h) }
        case _: # f is atomic
            return { frozenset({f}) }    

In [ ]:
f7

In [ ]:
f8 = cnf(f7)
f8

## Putting Everything Together

The function $f$ takes a <em style="color:blue">pure</em> formula $f$ from first order logic and transforms $f$ into a set of first order clauses.  Furthermore, $f$ **must not** contain free variables.

In [ ]:
def normalize(f):
    f1     = eliminateBiconditional(f)
    f2     = eliminateConditional(f1)
    f3     = nnf(f2)
    Qs, f4 = extractQuantifiers(f3)
    f5     = attachQuantifiers(Qs, f4)
    f6     = skolemize(f5, ())
    f7     = cnf(f6)
    return f7

In [ ]:
def test(s):
    f = LogicParser(s).parse()
    print(f'The knf of {s} is:')
    print(prettify(normalize(f)))

In [ ]:
def prettify(M):
    """Turn the set of frozen sets M into a string that looks like a set of sets.
       M is assumed to be a set of frozensets.
    """
    if M == set():
        return '{}'
    result = "{\n"
    for A in M:
        if A == frozenset(): 
            result += "{},\n"
        else:
            result += "    " + str(set(A)) + ",\n" # A is converted from a frozen set to a set
    result = result[:-2] # remove the trailing substring ", "
    result += "\n}"
    return result

In [ ]:
test(s)

In [ ]:
test('¬(∃y:∀x:P(x,y)→∀u:∃v:P(u,v))')